In [ ]:
from comparison_utils import *
from mpl_toolkits.mplot3d import Axes3D
import stream2 as st2
%load_ext rpy2.ipython

def load_slingshot(res_path,slingshot_ncenters):
    %R -i res_path -i slingshot_ncenters
    %R path <- slingshot_ncenters
    curves_gmm = %R readRDS(paste(res_path,'slingshot_curves_gmm.rds',sep='_'))
    curves_kmeans = %R readRDS(paste(res_path,'slingshot_curves_kmeans',path,'.rds',sep='_'))
    cl1 = %R readRDS(paste(res_path,'slingshot_gmm.rds',sep='_'))
    cl2 = %R readRDS(paste(res_path,'slingshot_kmeans',path,'.rds',sep='_'))
    
    all_lineages = {}
    for n,curve in zip(['kmeans','GMM'],[curves_kmeans,curves_gmm]):
        lineages = dict(curve.items())
        for k,v in lineages.items():
            lineages[k] = dict(v.items())
            for k2,v2 in lineages[k].items():
                lineages[k][k2]=np.array(lineages[k][k2])
            lineages[k]['s']=lineages[k]['s'][lineages[k]['ord']-1]
        all_lineages[n]=lineages

    all_lineages['kmeans'] = [v['s'] for k,v in all_lineages['kmeans'].items()]
    all_lineages['GMM'] = [v['s'] for k,v in all_lineages['GMM'].items()]
    return all_lineages

def load_monocle(res_path,mcle_ncenters ,mcle_sigma,mcle_gamma):
    %R -i res_path -i mcle_ncenters -i mcle_sigma -i mcle_gamma
    path = %R path <- paste(mcle_ncenters,mcle_sigma,mcle_gamma,sep='_')
    path=path[0]
    nodep = %R readRDS(paste(res_path,'monocle_dp_mst',path,'.rds',sep='_'))
    nodep = np.array(nodep.T)
    stree = scipy.io.mmread(res_path+f'_monocle_stree_{path}_.mm')
    edges = np.argwhere(np.triu(stree.todense()))
    clus = %R readRDS(paste(res_path,'monocle_clus',path,'.rds',sep='_'))
    partitions = np.array(np.array(clus)[1])
    clusters = np.array(np.array(clus)[2])
    return nodep, stree, edges, partitions, clusters

def run_monocle_slingshot_tuned(data_path, res_path, slingshot_ncenters = 50,mcle_ncenters = 50,mcle_sigma = 0.01,mcle_gamma = 0.5,mcle_eps = 1e-05):
    ! /home/jo/anaconda3/envs/qz_monocle3/bin/Rscript run_monocle_slingshot_tuned.R $data_path $slingshot_ncenters $mcle_ncenters $mcle_sigma $mcle_gamma $mcle_eps $res_path
    
    lineages = load_slingshot(res_path,slingshot_ncenters)
    nodep, stree, edges, partitions, clusters = load_monocle(res_path,mcle_ncenters ,mcle_sigma ,mcle_gamma)
    return lineages, nodep, stree, edges, partitions, clusters

def run_epg_disconnected(_anndata,n_nodes=None, n_clusters=20,epg_lambda=0.02,epg_mu=0.1,epg_alpha=0.02):
    X=_anndata.obsm['X_dr']

    kde = KernelDensity(kernel='gaussian',bandwidth=.5).fit(X)
    scores = kde.score_samples(X)
    scores = np.exp(scores)[:,None]
    pointweights = 1/scores
    pointweights /= np.sum(pointweights)  

    p_adatas = {}
    for clus in _anndata.obs['partition'].unique():
        p_adata =_anndata[_anndata.obs['partition']==clus].copy()
        p_pointweights = pointweights[_anndata.obs['partition']==clus]
        
        if n_nodes == None:
            p_n_nodes = max(50, int(np.ceil(np.sqrt(len(p_adata)))))
        else:
            p_n_nodes = n_nodes
            
        st2.tl.seed_graph(p_adata,obsm='X_dr',n_clusters=n_clusters)
        st2.tl.learn_graph(p_adata,obsm='X_dr',n_nodes=p_n_nodes,epg_alpha=epg_alpha,epg_mu=epg_mu,epg_lambda=epg_lambda,#GPU=True,
                                                    StoreGraphEvolution=True,PointWeights=p_pointweights,
                                                    MaxNumberOfGraphCandidatesDict={'AddNode2Node': 10, 'BisectEdge': 5, 'ShrinkEdge':np.inf},
                                                    #new kwargs
                                                    #epg_pseudotime=np.array(_anndata.X[:,0]+_anndata.X[:,1]),
                                                    #epg_pseudotimeLambda=0.005,
                                                    #epg_FixNodesAtPoints=[[1681]],#[[110]],#[[2445]],
                                                    verbose=1,
                                                    use_seed=True
                                                   )
        st2.tl.infer_pseudotime(p_adata,source=0)

        p_adatas[clus]=dict(epg_nodep=p_adata.uns['epg']['node_pos'],
                            epg_edges=p_adata.uns['epg']['edge'],
                            epg=nx.convert_matrix.from_scipy_sparse_matrix(p_adata.uns['epg']['conn']))

    return p_adatas, pointweights

In [ ]:
#%load_ext rpy2.ipython
#data_path = "/mnt/c/Users/jobac/Desktop/all/STREAM2/results/clus_tutorial_monocle_umap.rds"
#clus = %R readRDS('/mnt/c/Users/jobac/Desktop/all/STREAM2/results/clus_tutorial_monocle_clus.rds')
#celltype = %R readRDS("/mnt/c/Users/jobac/Desktop/all/STREAM2/final_notebooks/clus_tutorial_monocle_assigned_cell_type.rds")
#X_all = %R -i data_path readRDS(data_path)
#X_all_pca = %R readRDS("/mnt/c/Users/jobac/clus_tutorial_monocle_pca.rds")
#partition  = np.array(clus[1])-1
#big = np.where(np.bincount(partition)>500)[0]
#keep = np.isin(partition,big)
#partition = partition[keep]
#X = X_all[keep]
#X_pca = X_all_pca[keep]
#_anndata=sc.AnnData(scipy.sparse.csr_matrix((len(X_all),1)))
#_anndata.obsm['X_dr'] = _anndata.obsm['X_umap'] = X_all
#_anndata.obsm['X_pca'] = X_all_pca
#_anndata.obs['partition']=partition.astype(str)
#_anndata.obs['celltype']=np.array(celltype).astype(str)
##np.savetxt('/mnt/c/Users/jobac/Desktop/all/STREAM2/data/clus_tutorial_monocle_umap.txt',X.T.copy(),delimiter=',')
#data_path = "/mnt/c/Users/jobac/Desktop/all/STREAM2/data/clus_tutorial_monocle_umap.txt"

# Data

In [ ]:
%%R
library(monocle3)
library(dplyr) # imported for some downstream data manipulation

expression_matrix <- readRDS("/mnt/c/Users/jobac/Downloads/cao_l2_expression.rds")
cell_metadata <- readRDS("/mnt/c/Users/jobac/Downloads/cao_l2_colData.rds")
gene_annotation <- readRDS("/mnt/c/Users/jobac/Downloads/cao_l2_rowData.rds")

cds <- new_cell_data_set(expression_matrix,
                         cell_metadata = cell_metadata,
                         gene_metadata = gene_annotation)
cds <- preprocess_cds(cds, num_dim = 100)
cds = align_cds(cds, num_dim = 100, alignment_group = "plate")

cds <- reduce_dimension(cds)
cds = cluster_cells(cds, resolution=1e-5)
marker_test_res <- top_markers(cds, group_cells_by="partition", 
                               reference_cells=1000, cores=8)
top_specific_markers <- marker_test_res %>%
                            filter(fraction_expressing >= 0.10) %>%
                            group_by(cell_group) %>%
                            top_n(3, pseudo_R2)

top_specific_marker_ids <- unique(top_specific_markers %>% pull(gene_id))
colData(cds)$assigned_cell_type <- as.character(partitions(cds))
colData(cds)$assigned_cell_type = dplyr::recode(colData(cds)$assigned_cell_type,
                                                "1"="Germline",
                                                "2"="Body wall muscle",
                                                "3"="Unclassified neurons",
                                                "4"="Vulval precursors",
                                                "5"="Failed QC",
                                                "6"="Seam cells",
                                                "7"="Pharyngeal epithelia",
                                                "8"="Coelomocytes",
                                                "9"="Am/PH sheath cells",
                                                "10"="Failed QC",
                                                "11"="Touch receptor neurons",
                                                "12"="Intestinal/rectal muscle",
                                                "13"="Pharyngeal neurons",
                                                "14"="NA",
                                                "15"="flp-1(+) interneurons",
                                                "16"="Canal associated neurons",
                                                "17"="Ciliated sensory neurons",
                                                "18"="Other interneurons",
                                                "19"="Pharyngeal gland",
                                                "20"="Failed QC",
                                                "21"="Ciliated sensory neurons",
                                                "22"="Oxygen sensory neurons",
                                                "23"="Ciliated sensory neurons",
                                                "24"="Ciliated sensory neurons",
                                                "25"="Ciliated sensory neurons",
                                                "26"="Ciliated sensory neurons",
                                                "27"="Oxygen sensory neurons",
                                                "28"="Ciliated sensory neurons",
                                                "29"="Unclassified neurons",
                                                "30"="Socket cells",
                                                "31"="Failed QC",
                                                "32"="Pharyngeal gland",
                                                "33"="Ciliated sensory neurons",
                                                "34"="Ciliated sensory neurons",
                                                "35"="Ciliated sensory neurons",
                                                "36"="Failed QC",
                                                "37"="Ciliated sensory neurons",
                                                "38"="Pharyngeal muscle")
cds <- learn_graph(cds,learn_graph_control=list(ncenter = mcle_ncenters,L1.sigma = mcle_sigma,L1.gamma = mcle_gamma, eps = mcle_eps))
saveRDS(cds@principal_graph_aux@listData$UMAP$stree,'clus_tutorial_monocle_stree.rds')
saveRDS(cds@principal_graph_aux@listData$UMAP$Q,'clus_tutorial_monocle_Q.rds')
Matrix::writeMM(cds@principal_graph_aux@listData$UMAP$R,'clus_tutorial_monocle_R.mm')
saveRDS(cds@principal_graph_aux@listData$UMAP$dp_mst,'clus_tutorial_monocle_dp_mst.rds')
Matrix::writeMM(cds@principal_graph_aux@listData$UMAP$stree,'clus_tutorial_monocle_stree.mm')
saveRDS(reducedDim(cds, "UMAP"),"clus_tutorial_monocle_umap.rds")
saveRDS(cds@clusters$UMAP,'clus_tutorial_monocle_clus.rds')
saveRDS(cds@colData$assigned_cell_type,'clus_tutorial_monocle_assigned_cell_type.rds')

In [ ]:
data_path = "/mnt/c/Users/jobac/Desktop/all/STREAM2/results/clus_tutorial_monocle_umap.rds"
clus = %R readRDS('/mnt/c/Users/jobac/Desktop/all/STREAM2/results/clus_tutorial_monocle_clus.rds')
celltype = %R readRDS("/mnt/c/Users/jobac/Desktop/all/STREAM2/final_notebooks/clus_tutorial_monocle_assigned_cell_type.rds")
X_all = %R -i data_path readRDS(data_path)
X_all_pca = %R readRDS("/mnt/c/Users/jobac/clus_tutorial_monocle_pca.rds")
partition  = np.array(clus[1])-1
big = np.where(np.bincount(partition)>500)[0]
keep = np.isin(partition,big)
partition = partition[keep]
X = X_all[keep]
X_pca = X_all_pca[keep]
_anndata=sc.AnnData(X_pca)
_anndata.obsm['X_dr'] = _anndata.obsm['X_umap'] = X
_anndata.obsm['X_pca'] = X_pca
_anndata.obs['partition']=partition.astype(str)
_anndata.obs['celltype']=np.array(celltype)[keep].astype(str)
#np.savetxt('/mnt/c/Users/jobac/Desktop/all/STREAM2/data/clus_tutorial_monocle_umap.txt',X.T.copy(),delimiter=',')
data_path = "/mnt/c/Users/jobac/Desktop/all/STREAM2/data/clus_tutorial_monocle_umap.txt"

# Default trajectories

In [ ]:
#default results
(slingshot_lineages, 
 mcle_nodep, mcle_stree, mcle_edges, 
 mcle_partitions, mcle_clusters) = run_monocle_slingshot_tuned(data_path, slingshot_ncenters = 50, 
                                                         mcle_ncenters = 50, mcle_sigma = 0.01, 
                                                         mcle_gamma = 0.5, mcle_eps = 1e-05,res_path='R_outputs/clus_example/')

((via_out, via_projected_sc_pt, 
 (loci, c_edge, width_edge, pen_color, dot_size)),_, _) = run_VIA_disconnected(X,X,labels=None,root_user = [0], 
                                                            v0_toobig = 0.3 ,  random_seed = 42, 
                                                            knn = 20,ncomps=2)

((via_out_pca, via_projected_sc_pt_pca, 
 (loci_pca, c_edge_pca, width_edge_pca, pen_color_pca, dot_size_pca)),_,_,) = run_VIA_disconnected(_anndata.obsm['X_pca'],X,labels=None,root_user = [0], 
                                                            v0_toobig = 0.3 , random_seed = 42, 
                                                            knn = 20,ncomps=20)


paga_nodep, paga_edges, paga_weights = run_paga(_anndata,X,resolution=1,use_rep='X_dr')

p_adatas_epg, epg_pointweights = run_epg_disconnected(_anndata, n_nodes=None,epg_alpha=0.02)

for k,p_adata in p_adatas_epg.items():
    if max(np.array(list(nx.degree(p_adata['epg'])))[:,1].flatten())==2:
        p_adatas_epg[k]['merged_nodep'],p_adatas_epg[k]['merged_edges'] = None,None
        continue
    X_p =_anndata[_anndata.obs['partition']==k].obsm['X_dr'].copy()
    (_, _, _, 
     p_adatas_epg[k]['merged_nodep'],
     p_adatas_epg[k]['merged_edges']) = graph_loops.add_loops(X_p,p_adata['epg_nodep'],
                                                  p_adata['epg_edges'],
                                                  p_adata['epg'],
                                                  min_path_len=None,
                                                  nnodes=6,
                                                  max_inner_fraction=0.05,
                                                  min_node_n_points=5,
                                                  max_n_points=np.inf,
                                                  max_empty_curve_fraction=.8,
                                                  radius=None,
                                                  allow_same_branch=True,
                                                  fit_loops=True,
                                                  Lambda=0.02,
                                                  Mu=0.1,
                                                  weights=None,
                                                  plot=True,
                                                  verbose=True)

In [ ]:
p_adatas_epg, epg_pointweights = run_epg_disconnected(_anndata, n_clusters=30, n_nodes=None,epg_alpha=0.02)

for k,p_adata in p_adatas_epg.items():
    if max(np.array(list(nx.degree(p_adata['epg'])))[:,1].flatten())==2:
        p_adatas_epg[k]['merged_nodep'],p_adatas_epg[k]['merged_edges'] = None,None
        continue
    X_p =_anndata[_anndata.obs['partition']==k].obsm['X_dr'].copy()
    (_, _, _, 
     p_adatas_epg[k]['merged_nodep'],
     p_adatas_epg[k]['merged_edges']) = graph_loops.add_loops(X_p,p_adata['epg_nodep'],
                                                  p_adata['epg_edges'],
                                                  p_adata['epg'],
                                                  min_path_len=None,
                                                  nnodes=6,
                                                  max_inner_fraction=0.05,
                                                  min_node_n_points=5,
                                                  max_n_points=np.inf,
                                                  max_empty_curve_fraction=.8,
                                                  radius=None,
                                                  allow_same_branch=True,
                                                  fit_loops=True,
                                                  Lambda=0.02,
                                                  Mu=0.1,
                                                  weights=None,
                                                  plot=True,
                                                  verbose=True)

In [ ]:
for k,p_adata in p_adatas_epg.items():
    if max(np.array(list(nx.degree(p_adata['epg'])))[:,1].flatten())==2:
        p_adatas_epg[k]['merged_nodep'],p_adatas_epg[k]['merged_edges'] = None,None
        continue
    X_p =_anndata[_anndata.obs['partition']==k].obsm['X_dr'].copy()
    (_, _, _, 
     p_adatas_epg[k]['merged_nodep'],
     p_adatas_epg[k]['merged_edges']) = graph_loops.add_loops(X_p,p_adata['epg_nodep'],
                                                  p_adata['epg_edges'],
                                                  p_adata['epg'],
                                                  min_path_len=None,
                                                  nnodes=6,
                                                  max_inner_fraction=0.3,
                                                  min_node_n_points=5,
                                                  max_n_points=len(X_p)//50,
                                                  max_empty_curve_fraction=.8,
                                                  radius=None,
                                                  allow_same_branch=True,
                                                  fit_loops=True,
                                                  Lambda=0.02,
                                                  Mu=0.1,
                                                  weights=None,
                                                  plot=True,
                                                  verbose=True)

In [ ]:
for k,p_adata in p_adatas_epg.items():
    if max(np.array(list(nx.degree(p_adata['epg'])))[:,1].flatten())==2:
        p_adatas_epg[k]['merged_nodep'],p_adatas_epg[k]['merged_edges'] = None,None
        continue
    X_p =_anndata[_anndata.obs['partition']==k].obsm['X_dr'].copy()
    (_, _, _, 
     p_adatas_epg[k]['merged_nodep'],
     p_adatas_epg[k]['merged_edges']) = graph_loops2.add_loops(X_p,p_adata['epg_nodep'],
                                                  p_adata['epg_edges'],
                                                  p_adata['epg'],
                                                  min_path_len=None,
                                                  nnodes=6,
                                                  max_inner_fraction=0.3,
                                                  min_node_n_points=5,
                                                  max_n_points=len(X_p)//20,
                                                  max_empty_curve_fraction=.8,
                                                  radius=None,
                                                  allow_same_branch=True,
                                                  fit_loops=True,
                                                  Lambda=0.02,
                                                  Mu=0.1,
                                                  weights=None,
                                                  plot=True,
                                                  verbose=True)

In [ ]:
plt.rcParams.update({'font.size': 20})
fig, axs = plt.subplots(1,5,figsize=(30,5),sharey=True,sharex=True)
axs=axs.flat

X_plot=X#[idx]
c_plot=None#[idx]
linewidth=2

def labels_to_color(labels):
    '''Turn a list of labels into a list of colors'''
    import seaborn as sns
    import matplotlib.patches as mpatches
    unique_labels = np.unique(labels)
    colors = sns.color_palette('hls', len(unique_labels))

    # Associate each sample with a color indicating its type
    idx = np.zeros(len(labels)).astype(int)
    legend = []
    for i in range(len(unique_labels)):
        idx[labels == unique_labels[i]] = i
        legend.append(mpatches.Patch(color=colors[i], label=unique_labels[i]))

    col = [colors[i] for i in idx]
    return col, legend, unique_labels
c_plot, legend, unique_labels = labels_to_color(_anndata.obs['celltype'])

def init_subplot(d1=0,d2=2):
    ax = next(axs)
    ax.set_xlabel(f'Dim{d1+1}',labelpad=5)
    ax.set_ylabel(f'Dim{d2+1}',labelpad=5)
    ax.locator_params(axis='x',nbins=4)
    ax.locator_params(axis='y',nbins=4)
    ax.tick_params(axis="x",pad=10)
    ax.tick_params(axis="y",pad=-1)
    _=ax.scatter(X_plot[:,d1],X_plot[:,d2],c=c_plot,s=1,alpha=.2)
    return ax,_

for i0,(d1,d2) in enumerate(((0,1),)):

    #via
    ax,_=init_subplot(d1,d2)
    if i0==0: 
        ax.set_title('VIA')
        via_plot=via_out_pca
        (loci_plot, c_edge_plot, width_edge_plot, pen_color_plot, dot_size_plot)=(loci_pca, c_edge_pca, width_edge_pca,  pen_color_pca, dot_size_pca)
    else: 
        via_plot=via_out3
        (loci_plot, c_edge_plot, width_edge_plot, pen_color_plot, dot_size_plot)=(loci3, c_edge3, width_edge3, pen_color3, dot_size3)
    for i in range(len(via_plot)): 
        ax.plot(via_plot[i][0], via_plot[i][1], linewidth=linewidth, c='#323538')#1.5
    
        direction_arrow=via_plot[i][2]; head_width=via_out[i][3]
        if direction_arrow == 1:
            ax.arrow(via_plot[i][4], via_plot[i][5], via_plot[i][6], via_plot[i][7], shape='full', lw=0, length_includes_head=False,
                      head_width=.03, color='#323538')  
    count_ = 0
    for i, c_e, w, pc, dsz in zip(loci_plot, c_edge_plot, width_edge_plot, pen_color_plot, dot_size_plot):  # sc_supercluster_nn
        ax.scatter(X[i, d1], X[i, d2], c='black', s=dsz, edgecolors=c_e, linewidth=w)
        count_ = count_ + 1

    #monocle
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('Monocle3')
    for e in mcle_edges:
        curve_i = np.concatenate((mcle_nodep[[e[0]]], mcle_nodep[[e[1]]]),axis=0).T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)

    #slingshot
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('Slingshot')
    for l in slingshot_lineages['GMM']:
        curve_i = l.T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)

    #paga
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('PAGA')
    for e,w in zip(paga_edges,paga_weights):
        curve_i = np.concatenate((paga_nodep[[e[0]]], paga_nodep[[e[1]]]),axis=0).T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=2*linewidth*w)

    #epg
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('STREAM2')
    for k,p_adata in p_adatas_epg.items():
        if p_adatas_epg[k]['merged_edges'] is not None:
            _='merged_'
        else:
            _='epg_'
        for edge_i in p_adatas_epg[k][f'{_}edges']:
            curve_i = np.concatenate((p_adatas_epg[k][f'{_}nodep'][[edge_i[0]]], 
                                      p_adatas_epg[k][f'{_}nodep'][[edge_i[1]]]),axis=0).T
            ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)

#fig.tight_layout()
fig.subplots_adjust(right=0.85)
fig.legend(legend, unique_labels, loc = 'center right')

plt.savefig('figures/clus_example_default_2d.png',dpi=300,bbox_inches='tight')

In [ ]:
plt.rcParams.update({'font.size': 20})
fig, axs = plt.subplots(1,5,figsize=(30,5),sharey=True,sharex=True)
axs=axs.flat


def labels_to_color(labels):
    '''Turn a list of labels into a list of colors'''
    import seaborn as sns
    import matplotlib.patches as mpatches
    unique_labels = np.unique(labels)
    colors = sns.color_palette('hls', len(unique_labels))

    # Associate each sample with a color indicating its type
    idx = np.zeros(len(labels)).astype(int)
    legend = []
    for i in range(len(unique_labels)):
        idx[labels == unique_labels[i]] = i
        legend.append(mpatches.Patch(color=colors[i], label=unique_labels[i]))

    col = [colors[i] for i in idx]
    return col, legend, unique_labels
c_plot, legend, unique_labels = labels_to_color(_anndata.obs['celltype'])

def init_subplot(d1=0,d2=2):
    ax = next(axs)
    ax.set_xlabel(f'Dim{d1+1}',labelpad=5)
    ax.set_ylabel(f'Dim{d2+1}',labelpad=5)
    ax.locator_params(axis='x',nbins=4)
    ax.locator_params(axis='y',nbins=4)
    ax.tick_params(axis="x",pad=10)
    ax.tick_params(axis="y",pad=-1)
    _=ax.scatter(X_plot[:,d1],X_plot[:,d2],c=c_plot,s=1,alpha=.2)
    return ax,_
X_plot=X#[idx]
linewidth=2
ax,_=init_subplot(d1,d2)
if i0==0: ax.set_title('STREAM2')
for k,p_adata in p_adatas_epg.items():
    if p_adatas_epg[k]['merged_edges'] is not None:
        _='merged_'
    else:
        _='epg_'
    for edge_i in p_adatas_epg[k][f'{_}edges']:
        curve_i = np.concatenate((p_adatas_epg[k][f'{_}nodep'][[edge_i[0]]], 
                                  p_adatas_epg[k][f'{_}nodep'][[edge_i[1]]]),axis=0).T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)
fig.legend(legend, unique_labels, loc = 'center right')
plt.savefig('figures/TEMP_clus_example_default_2d.png',dpi=300,bbox_inches='tight')

In [ ]:
plt.rcParams.update({'font.size': 20})
fig, axs = plt.subplots(1,5,figsize=(30,5),sharey=True,sharex=True)
axs=axs.flat

X_plot=X#[idx]
c_plot=None#[idx]
linewidth=2

def labels_to_color(labels):
    '''Turn a list of labels into a list of colors'''
    import seaborn as sns
    import matplotlib.patches as mpatches
    unique_labels = np.unique(labels)
    colors = sns.color_palette('hls', len(unique_labels))

    # Associate each sample with a color indicating its type
    idx = np.zeros(len(labels)).astype(int)
    legend = []
    for i in range(len(unique_labels)):
        idx[labels == unique_labels[i]] = i
        legend.append(mpatches.Patch(color=colors[i], label=unique_labels[i]))

    col = [colors[i] for i in idx]
    return col, legend, unique_labels
c_plot, legend, unique_labels = labels_to_color(_anndata.obs['celltype'])

def init_subplot(d1=0,d2=2):
    ax = next(axs)
    ax.set_xlabel(f'Dim{d1+1}',labelpad=5)
    ax.set_ylabel(f'Dim{d2+1}',labelpad=5)
    ax.locator_params(axis='x',nbins=4)
    ax.locator_params(axis='y',nbins=4)
    ax.tick_params(axis="x",pad=10)
    ax.tick_params(axis="y",pad=-1)
    _=ax.scatter(X_plot[:,d1],X_plot[:,d2],c=c_plot,s=1,alpha=.2)
    return ax,_

for i0,(d1,d2) in enumerate(((0,1),)):

    #via
    ax,_=init_subplot(d1,d2)
    if i0==0: 
        ax.set_title('VIA')
        via_plot=via_out_pca
        (loci_plot, c_edge_plot, width_edge_plot, pen_color_plot, dot_size_plot)=(loci_pca, c_edge_pca, width_edge_pca,  pen_color_pca, dot_size_pca)
    else: 
        via_plot=via_out3
        (loci_plot, c_edge_plot, width_edge_plot, pen_color_plot, dot_size_plot)=(loci3, c_edge3, width_edge3, pen_color3, dot_size3)
    for i in range(len(via_plot)): 
        ax.plot(via_plot[i][0], via_plot[i][1], linewidth=linewidth, c='#323538')#1.5
    
        direction_arrow=via_plot[i][2]; head_width=via_out[i][3]
        if direction_arrow == 1:
            ax.arrow(via_plot[i][4], via_plot[i][5], via_plot[i][6], via_plot[i][7], shape='full', lw=0, length_includes_head=False,
                      head_width=.03, color='#323538')  
    count_ = 0
    for i, c_e, w, pc, dsz in zip(loci_plot, c_edge_plot, width_edge_plot, pen_color_plot, dot_size_plot):  # sc_supercluster_nn
        ax.scatter(X[i, d1], X[i, d2], c='black', s=dsz, edgecolors=c_e, linewidth=w)
        count_ = count_ + 1

    #monocle
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('Monocle3')
    for e in mcle_edges:
        curve_i = np.concatenate((mcle_nodep[[e[0]]], mcle_nodep[[e[1]]]),axis=0).T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)

    #slingshot
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('Slingshot')
    for l in slingshot_lineages['GMM']:
        curve_i = l.T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)

    #paga
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('PAGA')
    for e,w in zip(paga_edges,paga_weights):
        curve_i = np.concatenate((paga_nodep[[e[0]]], paga_nodep[[e[1]]]),axis=0).T
        ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=2*linewidth*w)

    #epg
    ax,_=init_subplot(d1,d2)
    if i0==0: ax.set_title('STREAM2')
    for k,p_adata in p_adatas_epg.items():
        if p_adatas_epg[k]['merged_edges'] is not None:
            _='merged_'
        else:
            _='epg_'
        for edge_i in p_adatas_epg[k][f'{_}edges']:
            curve_i = np.concatenate((p_adatas_epg[k][f'{_}nodep'][[edge_i[0]]], 
                                      p_adatas_epg[k][f'{_}nodep'][[edge_i[1]]]),axis=0).T
            ax.plot(curve_i[d1], curve_i[d2], c='k',linewidth=linewidth)

#fig.tight_layout()
fig.subplots_adjust(right=0.85)
fig.legend(legend, unique_labels, loc = 'center right')

plt.savefig('figures/clus_example_default_2d.png',dpi=300,bbox_inches='tight')

# Tuned

In [ ]:
# monocle, slingshot
for ncenters in (50,100,200):
    for mcle_sigma in (.00001,.0001,.001,.01,.1,1):
        for mcle_gamma in (.01,.1,.5,1,10):
            res_path='R_outputs/clus_example/'
            print(ncenters,mcle_sigma,mcle_gamma)
            if f'_monocle_dp_mst_{ncenters}_{mcle_sigma}_{mcle_gamma}_.rds' in os.listdir(res_path):
                continue
            try:
                run_monocle_slingshot_tuned(data_path, res_path=res_path,
                                            slingshot_ncenters = ncenters, mcle_ncenters = ncenters, 
                                            mcle_sigma = mcle_sigma, mcle_gamma = mcle_gamma, mcle_eps = 1e-05)
            except:
                pass

In [ ]:
#paga, via, epg
all_via_out = {}
for knn in [15,20,30,50,100]:
    for dist_std_local in [.5,1,2,5,10]:
        for jac_std_global in [.05,.1,.15,.3,.5,1]:
            ((via_out, via_projected_sc_pt, 
             (loci, c_edge, width_edge, pen_color, dot_size)),
             (via_out2, via_projected_sc_pt2,
             (loci2, c_edge2, width_edge2, pen_color2, dot_size2)),
             (via_out3, via_projected_sc_pt3,
             (loci3, c_edge3, width_edge3, pen_color3, dot_size3)))  = run_VIA(
                X,X,labels=None,  root_user = [0], v0_too_big = 0.3 , v1_too_big = 0.1, v0_random_seed = 42, 
                knn = knn,ncomps=3,jac_std_global=jac_std_global,dist_std_local=dist_std_local)
            all_via_out[(knn,dist_std_local,jac_std_global)]=[via_out, via_projected_sc_pt, (loci, c_edge, width_edge, pen_color, dot_size)]

all_paga_out = {}
for resolution in [.1,.2,.5,1,1.2,1.5,2,3]:
    for knn in [15,20,30,50,100]:
        paga_nodep, paga_edges, paga_clus_points = run_paga(_anndata,X,resolution=resolution)
        all_paga_out[(knn,resolution)]=[paga_nodep, paga_edges, paga_clus_points]
            
with open('all_via_out_clus_example.pkl','wb') as f:
    pickle.dump(all_via_out,f)
with open('all_paga_out_clus_example.pkl','wb') as f:
    pickle.dump(all_paga_out,f)

In [ ]:
import pickle

all_via_out_pca = {}
for knn in [15,20,30,50,100]:
    for dist_std_local in [1]:
        for jac_std_global in [.05,.1,.15,.3,.5,1]:
            print(f'all_via_out_clus_example_pca_{knn}_{dist_std_local}_{jac_std_global}')
            if f'all_via_out_clus_example_pca_{knn}_{dist_std_local}_{jac_std_global}.pkl' in os.listdir('../results/'):
                continue
            ((via_out, via_projected_sc_pt, 
             (loci, c_edge, width_edge, pen_color, dot_size)), _, _)  = run_VIA_disconnected(
                _anndata.obsm['X_pca'],X,labels=None,  root_user = [0], v0_toobig = 0.3 , random_seed = 42, 
                knn = knn,ncomps=20,jac_std_global=jac_std_global)
            all_via_out_pca[(knn,dist_std_local,jac_std_global)]=[via_out, via_projected_sc_pt, (loci, c_edge, width_edge, pen_color, dot_size)]
            with open(f'../results/all_via_out_clus_example_pca_{knn}_{dist_std_local}_{jac_std_global}.pkl','wb') as f:
                pickle.dump([via_out, via_projected_sc_pt, (loci, c_edge, width_edge, pen_color, dot_size)],f)